In [6]:
from qiskit.circuit.library import ZZFeatureMap,TwoLocal

In [7]:
def encode_circ(qc,num_features = 3):
    qc = qc + ZZFeatureMap(num_features)
    return qc

In [ ]:
def entangle_circ(qc, num_features = 3):
    qc = qc + TwoLocal(num_features * 2,reps = 1,rotation_blocks = 'ry',entanglement_blocks = 'cx',skip_final_rotation_layer = True)
    return qc

In [ ]:
ex_cir = 